In [1]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import pyLDAvis
import pyLDAvis.sklearn
from gensim.models.coherencemodel import CoherenceModel
from gensim.corpora import Dictionary
from gensim.models.ldamodel import LdaModel
from gensim.matutils import Sparse2Corpus

pyLDAvis.enable_notebook()
nltk.download('wordnet')
nltk.download('stopwords')
#sklearn.decomposition.LatentDirichletAllocation

[nltk_data] Downloading package wordnet to /home/anasofia/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/anasofia/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
data = pd.read_csv('data.csv')
data.fillna('', inplace=True)
data.head()

,contest,problem_name,problem_statement,problem_tags
0,325,A,You are given n rectangles. The corners of rec...,"implementation,*1500"
1,325,B,Daniel is organizing a football tournament. He...,"binarysearch,math,*1800"
2,325,C,Piegirl has found a monster and a book about m...,"dfsandsimilar,graphs,shortestpaths,*2600"
3,325,D,"In a far away land, there exists a planet shap...","dsu,*2900"
4,325,E,Piegirl found the red button. You have one las...,"combinatorics,dfsandsimilar,dsu,graphs,greedy,..."


In [3]:
print(f'quantidade de problemas: {data.shape[0]}')
print(f'quantidade de contests: {len(data.contest.unique())}')
print(f'problemas repetidos: {data.shape[0] - len(data.problem_statement.unique())}')

data.drop_duplicates(subset='problem_statement', inplace=True)

quantidade de problemas: 8343
quantidade de contests: 1424
problemas repetidos: 1524


## Pre-processamento

In [5]:
sw = stopwords.words('english')
sw += ['input', 'output', 'th', 'print', 'test', 'case', 'line']
lemma = WordNetLemmatizer()
text = data.problem_statement

# Problem_statement
print(text[8341])
print('------------')
text = text.apply(lambda x: re.sub(r'\${1,3}(.*?)\${1,3}', ' ',x))
text = text.apply(lambda x: re.sub(r'(?:\\)[a-zA-Z]+', ' ',x))
text = text.apply(lambda x: re.sub(r'\{(.*?)\}', ' ',x))
text = text.apply(lambda x: re.sub(r"[\,|\.|\(|\)|\!|\?|\:|\']",' ',x)) # removendo caracteres especiais
text = text.apply(lambda x: [i for i in x.split() if i.lower() not in sw]) # removendo stop-words
text = text.apply(lambda x: ' '.join([lemma.lemmatize(i) for i in x])) # lemmatizando todo

# Topics
topics = data.problem_tags
topics = topics.apply(lambda x: re.sub('[*][0-9]+','',x)) # remove os ratings *800 etc
topics = topics.apply(lambda x: [i for i in x.split(',') if i != '']) # tira strings vazias

print(text[8341])


At first, let's define function $$$f(x)$$$ as follows: $$$$$$ \begin{matrix} f(x) & = & \left\{ \begin{matrix} \frac{x}{2} & \mbox{if } x \text{ is even} \\ x - 1 & \mbox{otherwise } \end{matrix} \right. \end{matrix} $$$$$$

We can see that if we choose some value $$$v$$$ and will apply function $$$f$$$ to it, then apply $$$f$$$ to $$$f(v)$$$, and so on, we'll eventually get $$$1$$$. Let's write down all values we get in this process in a list and denote this list as $$$path(v)$$$. For example, $$$path(1) = [1]$$$, $$$path(15) = [15, 14, 7, 6, 3, 2, 1]$$$, $$$path(32) = [32, 16, 8, 4, 2, 1]$$$.

Let's write all lists $$$path(x)$$$ for every $$$x$$$ from $$$1$$$ to $$$n$$$. The question is next: what is the maximum value $$$y$$$ such that $$$y$$$ is contained in at least $$$k$$$ different lists $$$path(x)$$$?

Formally speaking, you need to find maximum $$$y$$$ such that $$$\left| \{ x ~|~ 1 \le x \le n, y \in path(x) \} \right| \ge k$$$.

The first line contains two integers $$$n$$$ an

In [6]:
text

0       given n rectangle corner rectangle integer coo...
1       Daniel organizing football tournament come fol...
2       Piegirl found monster book monster pie reading...
3       far away land exists planet shaped like cylind...
4       Piegirl found red button one last chance chang...
                              ...                        
8338    block arranged row numbered left right startin...
8339    map capital Berland viewed infinite coordinate...
8340    play strategic video game yeah ran good proble...
8341    first let define function follows f x & = & \ ...
8342    Recently lot student enrolled Berland State Un...
Name: problem_statement, Length: 6819, dtype: object

In [7]:
topics

0                                        [implementation]
1                                    [binarysearch, math]
2                  [dfsandsimilar, graphs, shortestpaths]
3                                                   [dsu]
4       [combinatorics, dfsandsimilar, dsu, graphs, gr...
                              ...                        
8338                                       [greedy, math]
8339       [bruteforce, geometry, greedy, implementation]
8340    [datastructures, dp, greedy, implementation, s...
8341              [binarysearch, combinatorics, dp, math]
8342                                         [bruteforce]
Name: problem_tags, Length: 6819, dtype: object

In [8]:
lista = topics.to_list()

d = dict()
for x in lista:
    for y in x:
        if y not in d:
            d[y]= 1
        else:
            d[y]+=1

x= sorted(d.items(), key=lambda item:item[1], reverse=True)

for i in x:
    print(i)



('implementation', 1970)
('math', 1733)
('greedy', 1645)
('dp', 1373)
('datastructures', 1076)
('bruteforce', 1020)
('constructivealgorithms', 981)
('graphs', 720)
('sortings', 658)
('binarysearch', 652)
('dfsandsimilar', 590)
('trees', 511)
('strings', 484)
('numbertheory', 453)
('combinatorics', 390)
('*specialproblem', 367)
('twopointers', 310)
('bitmasks', 300)
('geometry', 291)
('dsu', 210)
('shortestpaths', 174)
('divideandconquer', 167)
('probabilities', 162)
('hashing', 136)
('games', 131)
('interactive', 114)
('flows', 99)
('matrices', 87)
('stringsuffixstructures', 64)
('fft', 56)
('graphmatchings', 54)
('ternarysearch', 40)
('meet-in-the-middle', 33)
('expressionparsing', 32)
('2-sat', 19)
('chineseremaindertheorem', 10)
('schedules', 5)


## Vetorizando

In [9]:
max_df = 0.8
min_df = 0.05
vec = CountVectorizer(max_df=max_df, min_df=min_df, ngram_range=(1,3))

In [10]:
def vect2gensim(vectorizer, dtmatrix):
    # transform sparse matrix into gensim corpus and dictionary
    corpus_vect_gensim = Sparse2Corpus(dtmatrix, documents_columns=False)
    dictionary = Dictionary.from_corpus(corpus_vect_gensim,
            id2word=dict((id, word) for word, id in vectorizer.vocabulary_.items()))
    return (corpus_vect_gensim, dictionary)



In [11]:
X = vec.fit_transform(text)
X.shape
X = vect2gensim(vec, X)

## Agrupando

In [12]:
# Pegando quantidade de topicos

total_topics = set()
topics = topics.apply(lambda x: [total_topics.add(i) for i in x])

In [13]:
n_total_topics = len(total_topics)
print(f'Número de Tópicos: {n_total_topics}')
total_topics

Número de Tópicos: 37


{'*specialproblem',
 '2-sat',
 'binarysearch',
 'bitmasks',
 'bruteforce',
 'chineseremaindertheorem',
 'combinatorics',
 'constructivealgorithms',
 'datastructures',
 'dfsandsimilar',
 'divideandconquer',
 'dp',
 'dsu',
 'expressionparsing',
 'fft',
 'flows',
 'games',
 'geometry',
 'graphmatchings',
 'graphs',
 'greedy',
 'hashing',
 'implementation',
 'interactive',
 'math',
 'matrices',
 'meet-in-the-middle',
 'numbertheory',
 'probabilities',
 'schedules',
 'shortestpaths',
 'sortings',
 'strings',
 'stringsuffixstructures',
 'ternarysearch',
 'trees',
 'twopointers'}

In [14]:
n_components = n_total_topics
max_iter = 100

id2word = X[1]
corpus = X[0]

lda = LdaModel(corpus=corpus, num_topics=n_components, id2word=id2word, iterations=max_iter)

In [15]:
# lda_matrix = lda.fit_transform(X)



In [16]:
# top_word = lda.components_
# terms = vec.get_feature_names_out()

# for i, comp in enumerate(top_word):
#     top_terms_key = zip(terms, comp)
#     top_terms_key= sorted(top_terms_key, key = lambda t: t[1], reverse=True)[:10]
#     top_terms_list= list(dict(top_terms_key).keys())
#     print("Topic "+str(i)+": ",top_terms_list)

## Validando

In [17]:
new_model = CoherenceModel(model=lda, texts=text, corpus=corpus, dictionary=id2word, coherence="u_mass")
print(new_model.get_coherence())

# todo -> entender metrica de coerencia
#      -> usar dentro dos atributos o que ja foi calculado no CountVectorizer

-1.2191553189221824


## Visualizando

In [18]:
import pyLDAvis
import pyLDAvis.gensim
pyLDAvis.enable_notebook()

panel = pyLDAvis.gensim.prepare(lda, corpus, dictionary=id2word)
panel
# todo 
# mudar para o modelo do gensim

/home/anasofia/Documents/UnB/PIBIC/classificador-cf/env/lib/python3.8/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
0      0.051365  0.134209       1        1  4.787911
15    -0.057125  0.026024       2        1  4.377052
30     0.034301  0.060315       3        1  4.313980
35     0.044360 -0.020603       4        1  3.872041
14     0.038984 -0.071555       5        1  3.842385
32     0.000835 -0.046013       6        1  3.836602
25     0.007645  0.040264       7        1  3.775483
36    -0.113359 -0.015134       8        1  3.680854
18    -0.177888 -0.001435       9        1  3.623675
21     0.016612 -0.009734      10        1  3.606954
12    -0.040096 -0.023725      11        1  3.481805
19    -0.109099 -0.005909      12        1  3.371107
16     0.077300 -0.071219      13        1  3.191516
20     0.098630 -0.014894      14        1  3.175372
17    -0.120796  0.004291      15        1  2.885846
8     -0.133358 -0.045769      16        1  2.820854
4      0.067474  0.018965      17        1  2.706489
11    -0.024284 -0.058092      18        1  2.672269
26    -0.068340  0.018885      19        1  2.501949
3     -0.013671 -0.038656      20        1  2.473874
7      0.075186  0.036674      21        1  2.443837
22     0.049472 -0.009314      22        1  2.369808
31    -0.041740 -0.026729      23        1  2.260699
10    -0.041566  0.025449      24        1  2.220439
6      0.061798  0.000761      25        1  2.009255
24     0.084908  0.045967      26        1  1.996218
33     0.030303  0.102606      27        1  1.946656
28     0.054612 -0.036859      28        1  1.862615
1      0.069328  0.012393      29        1  1.767032
5     -0.012337  0.066292      30        1  1.668475
9      0.055838 -0.249021      31        1  1.661907
23     0.040116 -0.083974      32        1  1.656369
34    -0.081927  0.000091      33        1  1.611345
29    -0.016233  0.114761      34        1  1.532139
13     0.018591 -0.051363      35        1  1.479472
27     0.011134  0.118964      36        1  1.277330
2      0.063026  0.053087      37        1  1.238388, topic_info=          Term         Freq        Total Category  logprob  loglift
218     string  6727.000000  6727.000000  Default  30.0000  30.0000
257      array  3920.000000  3920.000000  Default  29.0000  29.0000
281  operation  3402.000000  3402.000000  Default  28.0000  28.0000
4        point  4230.000000  4230.000000  Default  27.0000  27.0000
111   sequence  4352.000000  4352.000000  Default  26.0000  26.0000
..         ...          ...          ...      ...      ...      ...
0        given    69.227737  5468.294478  Topic37  -4.4621   0.0220
128       find    53.402310  3224.952339  Topic37  -4.7216   0.2905
171        let    44.000575  2027.729186  Topic37  -4.9153   0.5609
48    possible    48.163036  5128.303816  Topic37  -4.8249  -0.2766
195    element    44.144311  3125.596349  Topic37  -4.9120   0.1314

[1779 rows x 6 columns], token_table=      Topic      Freq  Term
term                       
309       1  0.014886   000
309       2  0.003721   000
309       3  0.024809   000
309       4  0.032252   000
309       5  0.071947   000
...     ...       ...   ...
59       33  0.004359  zero
59       34  0.082086  zero
59       35  0.003632  zero
59       36  0.002906  zero
59       37  0.002906  zero

[10796 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 16, 31, 36, 15, 33, 26, 37, 19, 22, 13, 20, 17, 21, 18, 9, 5, 12, 27, 4, 8, 23, 32, 11, 7, 25, 34, 29, 2, 6, 10, 24, 35, 30, 14, 28, 3])